In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import numpy as np
from datetime import date, datetime, timedelta
import yfinance as yf
import random
import pandas_datareader as pdr


wiki_page = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

data = wiki_page[0]

list = data.to_dict(orient='records')

def ChooseCompanyTickers(iterations):
    random_elements = []
    for i in range(iterations):
        random_element = random.choice(list)['Symbol']
        if random_element in random_elements:
            random_element = random.choice(list)['Symbol']
        random_elements.append(random_element)
    return random_elements

def CalculateReturns(companytickers, start, end):
    arithmetic_returns = 0
    weighted_geometric_returns = 0
    estimated_returns = []
    i = 0
    for ticker in companytickers:
        data = yf.download(ticker, start, end, auto_adjust=True)
        string_start = start.strftime("%Y")
        string_end = end.strftime("%Y")
        duration = int(string_end) - int(string_start)
        closevalue = data['Close']
        closevaluedic = closevalue.to_dict()[ticker]
        first_value = next(iter(closevaluedic.values()))
        last_value = next(reversed(closevaluedic.values()))
        investmentreturns = (last_value - first_value)/first_value
#        arithmetic_returns += investmentreturns * (weightsinpercentage[i]/100)
        geometric_returns = ((last_value/first_value)**(1/duration) - 1)
        estimated_returns.append(geometric_returns)
#        weighted_geometric_returns += geometric_returns * (weightsinpercentage[i]/100)
        i += 1
    return estimated_returns

start = datetime(2005,12,31)
end = datetime(2024,12,31)
#companytickers = ChooseCompanyTickers(2)
#companytickers = ['BBVA', 'SAN', 'TEF', 'AAPL']
companytickers = [
    "ACS.MC",  # ACS, Actividades de Construcción y Servicios
    "AENA.MC",  # Aena S.M.E.
    "AMS.MC",  # Amadeus IT Group
    "ANA.MC",  # Acciona
    "BBVA.MC",  # Banco Bilbao Vizcaya Argentaria
    "BKT.MC",  # Bankinter
    "CABK.MC",  # CaixaBank
    "CLNX.MC",  # Cellnex Telecom
    "COL.MC",  # Inmobiliaria Colonial
    "ELE.MC",  # Endesa
    "ENG.MC",  # Enagás
    "FER.MC",  # Ferrovial
    "GRF.MC",  # Grifols
    "IAG.MC",  # International Airlines Group
    "IBE.MC",  # Iberdrola
    "ITX.MC",  # Inditex
    "MAP.MC",  # Mapfre
    "MEL.MC",  # Meliá Hotels International
    "MRL.MC",  # Merlin Properties
    "NTGY.MC",  # Naturgy Energy Group
    "PHM.MC",  # PharmaMar
    "RED.MC",  # Red Eléctrica de España
    "REP.MC",  # Repsol
    "SAN.MC",  # Banco Santander
    "SLR.MC",  # Solaria Energía y Medio Ambiente
    "TEF.MC",  # Telefónica
    "VIS.MC",  # Viscofan
]
print(companytickers)
#weightsinpercentage = [10, 20, 30, 40, 0]
calculatereturns = CalculateReturns(companytickers, start, end)
print(calculatereturns)

In [ ]:
def CompanyCorrelation(companytickers, start, end):
    list = []
    for ticker in companytickers:
        data = yf.download(ticker, start, end, auto_adjust=True)
        list.append(data[('Close')])
    consolidated_data = pd.concat(list, axis=1)
    consolidated_data_daily_pct_change = consolidated_data.pct_change().dropna()
    consolidated_data_daily_pct_change.columns = companytickers
    consolidated_data_daily_pct_change.describe()
    for ticker in companytickers:
        standard_deviation = np.std(consolidated_data_daily_pct_change[ticker])
        print(ticker, "mean : ", np.mean(consolidated_data_daily_pct_change[ticker]), "std :" , standard_deviation)
        print(ticker, stats.ttest_1samp(consolidated_data_daily_pct_change[ticker], 0))
        consolidated_data_daily_pct_change[ticker].plot(kind='hist', bins=100, figsize=(12,6), grid=True, title=(""), density=True)
        plt.xlabel('Return')
        plt.ylabel('Frequency Distribution of Return')
    correlation = consolidated_data_daily_pct_change.corr()
    return correlation

companycorrelation = CompanyCorrelation(companytickers, start, end)
print(companycorrelation)
numpycompanycorrelation = companycorrelation.to_numpy()
print(numpycompanycorrelation)

In [ ]:
def CompanyStandardDeviation(companytickers, start, end):
    list = []
    standard_deviations = []
    for ticker in companytickers:
        data = yf.download(ticker, start, end, auto_adjust=True)
        list.append(data[('Close')])
    consolidated_data = pd.concat(list, axis=1)
    consolidated_data_daily_pct_change = consolidated_data.pct_change().dropna()
    consolidated_data_daily_pct_change.columns = companytickers
    consolidated_data_daily_pct_change.describe()
    for ticker in companytickers:
        standard_deviation = np.std(consolidated_data_daily_pct_change[ticker])
        standard_deviations.append(standard_deviation)
    return standard_deviations

companystandarddeviations = CompanyStandardDeviation(companytickers, start, end)
print(companystandarddeviations)

In [ ]:
def OverallStandardDeviation(comapanystandarddeviations, companycorrelation, weights):
    import numpy as np

    # Example: Standard deviations of 4 assets
    std_devs = np.array(comapanystandarddeviations)  # Replace with your values

    # Example: 4x4 Correlation Matrix (Replace with actual data)
    correlation_matrix = np.array(companycorrelation)

    # Compute the covariance matrix: Σ = D * P * D
    D = np.diag(std_devs)  # Create a diagonal matrix of standard deviations
    covariance_matrix = D @ correlation_matrix @ D  # Matrix multiplication

    # Example: Portfolio weights (replace with actual values)
    weights = np.array(weights) / 100  # Sum should be 1

    # Compute portfolio variance: σ_p^2 = w^T Σ w
    portfolio_variance = weights.T @ covariance_matrix @ weights

    # Compute portfolio standard deviation (σ_p)
    portfolio_std_dev = np.sqrt(portfolio_variance)

    print("Portfolio Standard Deviation (σ_p):", portfolio_std_dev)

    return portfolio_std_dev

#print(OverallStandardDeviation(companystandarddeviations, numpycompanycorrelation, weightsinpercentage))

In [ ]:
def get_risk_free_rate():
    syms = ['DGS10']
    ir_data = pdr.DataReader(syms, 'fred', datetime(2006, 12,1),datetime.today())
    riskfreeratechart = ir_data.iloc[[-1]]
    dictionary = riskfreeratechart.to_dict(orient='records')
    riskfreerate = dictionary[0]["DGS10"]
    riskfreerate = riskfreerate/100

    return riskfreerate 

In [ ]:
import mplcursors
import matplotlib.pyplot as plt


def EfficientFrontier(expectedreturns, standarddeviations, companycorrelation, risk_free_rate):
    i = len(expectedreturns)
    # Given expected returns of 4 stocks (Replace with actual values)
    expected_returns = np.array(expectedreturns)  # Example returns for 4 assets

    # Given standard deviations of 4 stocks
    std_devs = np.array(standarddeviations)

    # Given correlation matrix
    correlation_matrix = np.array(companycorrelation)

    # Compute covariance matrix
    D = np.diag(std_devs)
    covariance_matrix = D @ correlation_matrix @ D

    # Generate multiple weight combinations
    num_portfolios = 100000
    portfolio_returns = []
    portfolio_std_devs = []
    portfolio_sharpe_ratios = []
    portfolio_weights = []

    for _ in range(num_portfolios):
        weights = np.random.random(i)  # Generate random weights
        weights /= np.sum(weights)  # Normalize to sum to 1

        # Portfolio return
        portfolio_return = np.dot(weights, expected_returns)

        # Portfolio standard deviation
        portfolio_variance = weights.T @ covariance_matrix @ weights
        portfolio_std_dev = np.sqrt(portfolio_variance)

        portfolio_sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

        # Store results
        portfolio_returns.append(portfolio_return)
        portfolio_std_devs.append(portfolio_std_dev)
        portfolio_sharpe_ratios.append(portfolio_sharpe_ratio)
        portfolio_weights.append(weights)

    weight = [
    0.0207,  # ACS.MC
    0.0378,  # AENA.MC
    0.0480,  # AMS.MC
    0.0076,  # ANA.MC
    0.1083,  # BBVA.MC
    0.0123,  # BKT.MC
    0.0563,  # CABK.MC
    0.0357,  # CLNX.MC
    0.0021,  # COL.MC
    0.0133,  # ELE.MC
    0.0048,  # ENG.MC
    0.0464,  # FER.MC
    0.0066,  # GRF.MC
    0.0310,  # IAG.MC
    0.1301,  # IBE.MC
    0.1425,  # ITX.MC
    0.0073,  # MAP.MC
    0.0000,  # MEL.MC (Not in the provided list, assumed 0.0)
    0.0086,  # MRL.MC
    0.0071,  # NTGY.MC
    0.0000,  # PHM.MC (Not in the provided list, assumed 0.0)
    0.0137,  # RED.MC
    0.0212,  # REP.MC
    0.1381,  # SAN.MC
    0.0014,  # SLR.MC
    0.0357,  # TEF.MC
    0.0000,  # VIS.MC (Not in the provided list, assumed 0.0)
    ]

    weights = np.array(weight)

    # Portfolio return
    portfolio_return = np.dot(weights, expected_returns)

    # Portfolio standard deviation
    portfolio_variance = weights.T @ covariance_matrix @ weights
    portfolio_std_dev = np.sqrt(portfolio_variance)

    portfolio_sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

    # Store results
    portfolio_returns.append(portfolio_return)
    portfolio_std_devs.append(portfolio_std_dev)
    portfolio_sharpe_ratios.append(portfolio_sharpe_ratio)
    portfolio_weights.append(weights)

    position = portfolio_sharpe_ratios.index(max(portfolio_sharpe_ratios))
    portfolio_std_devs[position]
    portfolio_returns[position]
    portfolio_weights[position]

    # Plot the Efficient Frontier
    plt.figure(figsize=(10, 6))
    plt.scatter(portfolio_std_devs, portfolio_returns, c=portfolio_returns, marker='o')
    plt.scatter(portfolio_std_devs[position], portfolio_returns[position], c="red", label="Highlighted Portfolio", marker='x')
    plt.text(portfolio_std_devs[position], portfolio_returns[position], "  Optimal", fontsize=8, verticalalignment='bottom')
    plt.scatter(0, risk_free_rate, c="red", label="Risk Free", marker='x')
    plt.text(0, risk_free_rate, "  Risk Free", fontsize=8, verticalalignment='bottom')
    plt.scatter(portfolio_std_devs[-1], portfolio_returns[-1], c="red", label="Risk Free", marker='x')
    plt.text(portfolio_std_devs[-1], portfolio_returns[-1], "  ETF", fontsize=8, verticalalignment='bottom')
    plt.xlabel("Portfolio Standard Deviation (Risk)")
    plt.ylabel("Expected Portfolio Return")
    plt.title("Efficient Frontier")
    plt.show()
    return portfolio_std_devs, portfolio_returns, portfolio_weights, position, portfolio_weights[position], portfolio_sharpe_ratios

risk_free_rate = get_risk_free_rate()
efficientfrontier = EfficientFrontier(calculatereturns, companystandarddeviations, companycorrelation, risk_free_rate)
print(efficientfrontier)
print(companytickers)


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Example data (replace with your actual data)
portfolio_std_devs = efficientfrontier[0]  # Random standard deviations
portfolio_returns = efficientfrontier[1]  # Random returns
portfolio_weights = efficientfrontier[2]  # Portfolio weights
position = efficientfrontier[3]  # Example position of the optimal portfolio
portfolio_sharpe_ratio = efficientfrontier[5]
risk_free_rate = risk_free_rate  # Example risk-free rate

# Create the scatter plot
fig = go.Figure()

# Add the main scatter plot
fig.add_trace(go.Scatter(
    x=portfolio_std_devs,
    y=portfolio_returns,
    mode='markers',
    marker=dict(
        size=10,
        color=portfolio_returns,  # Color by returns
    ),
    name='Portfolios',
    hovertemplate=(
        "<b>Risk</b>: %{x:.4f}<br>"
        "<b>Return</b>: %{y:.4f}<br>"
        "<b>Weights</b>: %{customdata}<br>"  # Display portfolio weights
        "<extra></extra>"
    ),
    customdata = [
    f"{', '.join(f'{name}: {weight:.4f}' for name, weight in zip(companytickers, weights))}<br>"
    f"<b>Sharpe Ratio</b>: {sharpe_ratio:.4f}"
    for weights, sharpe_ratio in zip(portfolio_weights, portfolio_sharpe_ratio)
]
#    customdata=[", ".join(map(lambda x, name: f"{name} {x:.4f}", weights, companytickers)) for weights in portfolio_weights],  # Format weights as strings
))

for name, weight in zip(companytickers, portfolio_weights[position]):
    w = weight * 100
    print(f'{name}: {w:.4f}%')

# Add the highlighted portfolio
fig.add_trace(go.Scatter(
    x=[portfolio_std_devs[position]],
    y=[portfolio_returns[position]],
    mode='markers',
    marker=dict(
        size=12,
        color='red',
        symbol='x',
    ),
    name='Optimal Portfolio',
    hovertemplate=(
        "<b>Optimal Portfolio</b><br>"
        "<b>Risk</b>: %{x:.4f}<br>"
        "<b>Return</b>: %{y:.4f}<br>"
        "<b>Weights</b>: %{customdata}<br>"  # Display portfolio weights
        "<extra></extra>"
    ),
    customdata = [", ".join(map(lambda x, name: f"{name} {x:.4f}", portfolio_weights[position], companytickers)) + "<br><b>" + f"Sharpe Ratio: </b> {portfolio_sharpe_ratio[position]:.4f}"]
))

# Add the risk-free rate point
fig.add_trace(go.Scatter(
    x=[0],
    y=[risk_free_rate],
    mode='markers',
    marker=dict(
        size=12,
        color='red',
        symbol='x',
    ),
    name='Risk-Free Rate',
    hovertemplate=(
        "<b>Risk-Free Rate</b><br>"
        "<b>Risk</b>: %{x:.4f}<br>"
        "<b>Return</b>: %{y:.4f}<br>"
        "<extra></extra>"
    ),
))

# Update layout
fig.update_layout(
    title="Efficient Frontier",
    xaxis_title="Portfolio Standard Deviation (Risk)",
    yaxis_title="Expected Portfolio Return",
    hovermode='closest',  # Show hover info for the closest point
)

# Show the plot
fig.show()